# "Economic impact of the covid19 sectoral business closures"
> "the impact of sector closure on GPD"
-toc: true
-math: true
-badges: false

### Sophie Osotimehin and Latchezar Popov
##### Visualizations by Julien Duranton

In [1]:
#hide_input
import ipywidgets as widgets
import altair as alt
from IPython.display import display
from ipywidgets import interact
import pandas as pd
from vega_datasets import data as data_world
import sys
import os


root = "C:\\Users\\julie\\Documents\\GitHub\\test-blog\\input_data"

data_agg = pd.read_csv( root + '\\covid\\results_agg_downstreamshock.csv')
data_sec = pd.read_csv(root + "\\covid\\results_sec_downstreamshock.csv")
mapping_industry = pd.read_csv(root + "\\Indcode_mapping_full.csv")
country_codes = pd.read_csv(root + '\\countries_codes_and_coordinates.csv')

#rename columns
data_agg = data_agg.rename(columns={'indrecord':'Indcode', 'gdp':'GDP', 'wellfare':'Wellfare'})
data_agg['Indcode'] = [int(data_agg.Indcode[i].replace('c','')) 
                       if data_agg.Indcode[i] != 'all5' else data_agg.Indcode[i] for i in data_agg.index.tolist()]
mapping_industry = pd.concat([mapping_industry,pd.DataFrame([{'Name':'all5','Indcode':'all5'}])],ignore_index=True)
data_agg = data_agg.merge(mapping_industry, on='Indcode', how='left')

country_codes = country_codes.rename(columns={'Alpha-3 code':'country'})                                                                      
data_countries = pd.DataFrame(data_agg.country.unique(), columns=['country'])
data_countries = data_countries.merge(country_codes, on='country', how='left')

#format dataframe sectors
data_sec = data_sec.melt(['country','industry'], var_name='IndClosed', value_name='value')
data_sec = data_sec.rename(columns={'industry':'downstream'})
data_sec['IndClosed'] = [int(data_sec.IndClosed[i].replace('c','')) 
                       if data_sec.IndClosed[i] != 'all5' else data_sec.IndClosed[i] for i in data_sec.index.tolist()]


data_sec['Name'] = [mapping_industry[mapping_industry.Indcode == data_sec.IndClosed[i]].Name.values[0] 
                         for i in data_sec.index.tolist()]
data_sec['dsName'] = [mapping_industry[mapping_industry.Indcode == data_sec.downstream[i]].Name.values[0] 
                         for i in data_sec.index.tolist()]

In [2]:
#hide_input


source = alt.topo_feature(data_world.world_110m.url, 'countries')


alt.data_transformers.disable_max_rows()

background = alt.Chart(source).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=600,
    height=300
).project(type='equirectangular')

selector = alt.selection_single(empty='all', fields=['country'],bind='legend', init={'country':'FRA'})
selector_sector = alt.selection_single(empty='all', fields=['Name'],bind='legend', init={'Name':'all5'})


# Points and text
hover = alt.selection(type='single', on='mouseover', nearest=True,
                      fields=['Latitude (average)', 'Longitude (average)'])

base = alt.Chart(data_countries).encode(
    longitude='Longitude (average):Q',
    latitude='Latitude (average):Q',
)

text = base.mark_text(dy=-5, align='right').encode(
    alt.Text('country', type='nominal'),
    opacity=alt.condition(~selector, alt.value(0), alt.value(1))
)

points = base.mark_point().encode(
    color=alt.condition(selector, 'country:N', alt.value('lightgray'), 
                        legend=alt.Legend(orient='left', columns=1, symbolLimit=50), title='Select Country'),
    size=alt.condition(~hover, alt.value(30), alt.value(100))
).add_selection(hover).add_selection(selector)

# Define the 3 Charts

chart_1 = alt.Chart(data_agg, width=600, height=300).mark_point().encode(
    x='GDP:Q',
    y='Wellfare:Q',
    size=alt.condition(selector_sector, alt.value(100), alt.value(30)),
    tooltip=['country', 'Name', 'GDP', 'Wellfare'],
    color=alt.condition(selector_sector, 'Name:N', alt.value('lightgray'), 
                        legend = alt.Legend( columns=1, symbolLimit=50),
                        title='Select Sector to close')
    ).add_selection(selector).add_selection(selector_sector)

chart_2 = alt.Chart(data_sec, width=600, height=300).mark_bar().encode(
    y=alt.Y('dsName:N',sort=alt.SortField(field='value', order='descending'), 
            axis=alt.Axis(title='Sectors',orient='left',labelLimit=100,labelAlign='right'),
           ),
    x=alt.X('value:Q',stack=None),
    color=alt.condition(selector_sector, 'Name:N', alt.value('lightgray')),
    tooltip=['country','Name','dsName','value']).add_selection(selector)



filter_chart_1 = chart_1.transform_filter(
    selector).add_selection(
    selector_sector).properties(title="Loss of GPD and Welfare by Sector closed")

filter_chart_2 = chart_2.add_selection(selector).transform_filter(
    selector).add_selection(selector_sector).transform_filter(
    selector_sector).properties(title="Impact of Sector closure on downstream sectors")

(background + points + text).resolve_scale(color='independent') & (filter_chart_1 & filter_chart_2).resolve_scale(color='shared')


alt.VConcatChart(...)